In [311]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# %matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
# color = sns.color_palette()
# sns.set_style('darkgrid')

# import warnings
# def ignore_warn(*args, **kwargs):
#     pass
# warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

from scipy import stats
from scipy.stats import norm, skew #for some statistics

# pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8")) #check the files available in the directory

In [312]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [313]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
y = all_data['SalePrice']
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2919, 80)


/home/wkm/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [314]:
#############
# input missing value
#############

all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median())) ###
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [315]:
FireplaceQu_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
        'None':0
    }
all_data['FireplaceQu'] = all_data['FireplaceQu'].map(FireplaceQu_mapping)
BsmtQual_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
        'None':0
    }
all_data['BsmtQual'] = all_data['BsmtQual'].map(BsmtQual_mapping)
BsmtCond_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
        'None':0
    }
all_data['BsmtCond'] = all_data['BsmtCond'].map(BsmtCond_mapping)
GarageQual_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
        'None':0
    }
all_data['GarageQual'] = all_data['GarageQual'].map(GarageQual_mapping)
GarageCond_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
        'None':0
    }
all_data['GarageCond'] = all_data['GarageCond'].map(GarageCond_mapping)
ExterQual_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
    }
all_data['ExterQual'] = all_data['ExterQual'].map(ExterQual_mapping)
ExterCond_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
    }
all_data['ExterCond'] = all_data['ExterCond'].map(ExterCond_mapping)
HeatingQC_mapping = {
        'Ex':5,
        'Gd':4,
        'TA':3,
        'Fa':2,
        'Po':1,
    }
all_data['HeatingQC'] = all_data['HeatingQC'].map(HeatingQC_mapping)
PoolQC_mapping = {
        'Ex':4,
        'Gd':3,
        'TA':2,
        'Fa':1,
        'None':0
    }
all_data['PoolQC'] = all_data['PoolQC'].map(PoolQC_mapping)


KitchenQual_mapping = {
        'Ex':4,
        'Gd':3,
        'TA':2,
        'Fa':1,
        'Po':0,
    }
all_data['KitchenQual'] = all_data['KitchenQual'].map(KitchenQual_mapping)

BsmtFinType1_mapping = {
        'GLQ':6,
        'ALQ':5,
        'BLQ':4,
        'Rec':3,
        'LwQ':2,
        'Unf':1,
        'None':0,
    }
all_data['BsmtFinType1'] = all_data['BsmtFinType1'].map(BsmtFinType1_mapping)

BsmtFinType2_mapping = {
        'GLQ':6,
        'ALQ':5,
        'BLQ':4,
        'Rec':3,
        'LwQ':2,
        'Unf':1,
        'None':0,
    }
all_data['BsmtFinType2'] = all_data['BsmtFinType2'].map(BsmtFinType2_mapping)

Functional_mapping = {
        'Typ':7,
        'Min1':6,
        'Min2':5,
        'Mod':4,
        'Maj1':3,
        'Maj2':2,
        'Sev':1,
        'Sal':0,
    }
all_data['Functional'] = all_data['Functional'].map(Functional_mapping)

Fence_mapping = {
        'GdPrv':4,
        'MnPrv':3,
        'GdWo':2,
        'MnWw':1,
        'None':0,
    }
all_data['Fence'] = all_data['Fence'].map(Fence_mapping)

BsmtExposure_mapping = {
        'Gd':4,
        'Av':3,
        'Mn':2,
        'No':1,
        'None':0,
    }
all_data['BsmtExposure'] = all_data['BsmtExposure'].map(BsmtExposure_mapping)


all_data['GarageFinish'] = all_data['GarageFinish'].map({'None':0, 'Unf':1, 'RFn':2, 'Fin':3})
all_data['LandSlope'] = all_data['LandSlope'].map({'Sev':1, 'Mod':2, 'Gtl':3})
all_data['LotShape'] = all_data['LotShape'].map({'IR3':1, 'IR2':2, 'IR1':3, 'Reg':4})
all_data['PavedDrive'] = all_data['PavedDrive'].map({'N':1, 'P':2, 'Y':3})
all_data['Street'] = all_data['Street'].map({'Grvl':1, 'Pave':2})
all_data['Alley'] = all_data['Alley'].map({'None':0, 'Grvl':1, 'Pave':2})
all_data['CentralAir'] = all_data['CentralAir'].map({'N':0, 'Y':1})


In [316]:
# Check the skew of all numerical features

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness


Skew in numerical features: 



,Skew
MiscVal,21.947195
PoolQC,19.117440
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
KitchenAbvGr,4.302254
BsmtFinSF2,4.146143
Alley,4.138385
EnclosedPorch,4.003891


In [317]:
all_data.to_csv('../data/all_data_before_boxcox.csv', index=False)

from scipy import stats

skewness = skewness[abs(skewness) > 1].dropna()
print("There are {} skewed numerical features to Box-Cox transform".format(skewness.shape[0]))

skewed_features = skewness.index.drop(['LotArea','1stFlrSF','GrLivArea','Street'])

for feat in skewed_features:
    if min(all_data[feat]) <= 0:
        all_data[feat] = all_data[feat] + min(all_data[feat]) + 1
    all_data[feat] = stats.boxcox(all_data[feat])[0]
    
    
all_data.to_csv('../data/all_data_after_boxcox.csv', index=False)

There are 36 skewed numerical features to Box-Cox transform


In [318]:
# Getting dummy categorical features

all_data = pd.get_dummies(all_data)

In [319]:
# standardlize all explanatory variables
# for i in list(all_data):
#     all_data[i] = (all_data[i] - all_data[i].mean()) / all_data[i].std()

all_data['SalePrice'] = y
train = all_data[:ntrain]
test = all_data[ntrain:]

# Shuffle
# train = train.sample(frac=1).reset_index(drop=True)

In [320]:
# Getting the new train and test sets.

train.to_csv('../data/train_transformed.csv', index=False)
test.to_csv('../data/test_transformed.csv', index=False)

In [321]:
train

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SalePrice
0,856,854,0.000000,0.000000,3,28.417019,0.582325,15.033885,0.000000,6,...,0,0,0,0,0,0,0,0,1,208500.0
1,1262,0,0.000000,0.000000,3,28.417019,1.091682,16.516375,0.000000,5,...,0,0,0,0,0,0,0,0,1,181500.0
2,920,866,0.000000,0.000000,3,28.417019,0.837681,13.464526,0.000000,6,...,0,0,0,0,0,0,0,0,1,223500.0
3,961,756,0.000000,0.000000,3,59.390567,0.582325,10.482283,0.000000,5,...,0,0,0,0,0,0,0,0,1,140000.0
4,1145,1053,0.000000,0.000000,4,28.417019,0.988939,14.708008,0.000000,6,...,0,0,0,0,0,0,0,0,1,250000.0
5,796,566,0.066197,0.000000,1,28.417019,0.582325,15.193056,0.000000,6,...,0,0,0,0,0,0,0,0,1,143000.0
6,1694,0,0.000000,0.000000,3,28.417019,0.988939,18.165358,0.000000,6,...,0,0,0,0,0,0,0,0,1,307000.0
7,1107,983,0.000000,0.000000,3,28.417019,0.837681,15.913051,0.679706,5,...,0,0,0,0,0,0,0,0,1,200000.0
8,1022,752,0.000000,0.000000,2,28.417019,0.582325,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0,1,129900.0
9,1077,0,0.000000,0.000000,2,28.417019,0.582325,15.870221,0.000000,6,...,0,0,0,0,0,0,0,0,1,118000.0
